## Lid Driven Cavity

In [233]:
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum


## Definitions

In [234]:
GRID_SIZE = (41,41)
DOMAIN_SIZE = (1,1) # m

rho = 1 # kg/m^3 Density
nu = 0.01 # m/s^2 Kinematic Viscosity (v -> nu)

h = (DOMAIN_SIZE[0] / GRID_SIZE[0])# m : size of one cell
delta_t = 1 # s

In [235]:
class BCType(Enum):
    DIRICHLET = 0
    NEUMANN = 1

class BCLocation(Enum):
    """
    Enum for defining the location where bc is applied
    by calling .value you can get the slice for the grid
    """
    TOP = (0, slice(None))         # [0, :]
    BOTTOM = (-1, slice(None))     # [-1, :]
    LEFT = (slice(None), 0)        # [:, 0]
    RIGHT = (slice(None), -1)      # [:, -1]
    
    # One row inside
    TOP_INSIDE = (1, slice(None))        # [1,:]
    BOTTOM_INSIDE = (-2, slice(None))    # [-2,:]
    LEFT_INSIDE = (slice(None), 1)       # [:, 1]
    RIGHT_INSIDE = (slice(None), -2)     # [:,-2]
    

    def inside(self):
        """
        Returns the row next to the boundary, that is inside the domain
        with .value you can get the slicing object

        Returns:
            BCLocation: The Location inside the domain next to the boundary
        """
        match self:
            case BCLocation.TOP:
                return BCLocation.TOP_INSIDE
            case BCLocation.BOTTOM:
                return BCLocation.BOTTOM_INSIDE
            case BCLocation.LEFT:
                return BCLocation.LEFT_INSIDE
            case BCLocation.RIGHT:
                return BCLocation.RIGHT_INSIDE 
            
        

class FluidProperty(Enum):
    VELOCITY = 0
    PRESSURE = 1

In [236]:
BCLocation.BOTTOM.inside().value

(-2, slice(None, None, None))

## 0. Initialisierung am Rechengitter

In [237]:
class Grid():
    def __init__(self, 
                 grid_size: tuple[int, int], 
                 domain_size: tuple[int,int]):
        
        # dimensions
        self.grid_size = grid_size
        self.domain_size = domain_size
        
        # points along one coordinate axis (maybe dont need to save)
        self._x_points: np.ndarray
        self._y_points: np.ndarray
        
        # meshgrid coordinates
        self.x: np.ndarray
        self.y: np.ndarray
        
        # velocity
        self.ux: np.ndarray
        self.uy: np.ndarray
        
        # pressure
        self.p: np.ndarray
        
        self._initialize_mesh()
        
        
    def _initialize_mesh(self) -> None:
        """
        Initializes the mesh
        Coordinates x, y + Grids for pressure, velocity in x and y direction
        """
        self._x_points = np.linspace(0,self.domain_size[0], self.grid_size[0])
        self._y_points = np.linspace(0, self.domain_size[1], self.grid_size[1])
        
        self.x, self.y = np.meshgrid(self._x_points, self._y_points) 
    
        self.ux = np.zeros(self.grid_size)
        self.uy = np.zeros(self.grid_size)
        self.p = np.zeros(self.grid_size)
            
    
    def bc_enforce_pressure(self) -> None:
        """
        Enforces the pressure boundary conditions on the grid
        """
        self._bc_enforce(type=BCType.NEUMANN, 
                        location=[BCLocation.RIGHT, BCLocation.BOTTOM, BCLocation.LEFT], 
                        val=0,
                        fluid_property=FluidProperty.PRESSURE)
        
        self._bc_enforce(type=BCType.DIRICHLET, 
                        location=BCLocation.TOP, 
                        val=0,
                        fluid_property=FluidProperty.PRESSURE)  
        
        
    def bc_enforce_velocity(self) -> None:
        """
        Enforces the velocity boundary conditions on the grid
        """
        self._bc_enforce(type=BCType.DIRICHLET, 
                        location=[BCLocation.RIGHT, BCLocation.BOTTOM, BCLocation.LEFT], 
                        val=[0,0],
                        fluid_property=FluidProperty.VELOCITY)
        
        self._bc_enforce(type=BCType.DIRICHLET,
                        location=BCLocation.TOP,
                        val=[1,0],
                        fluid_property=FluidProperty.VELOCITY)
        
        
    def _bc_enforce(self,
                   type: BCType,
                   location: BCLocation | list[BCLocation], 
                   val: list[float,float] | float,
                   fluid_property: FluidProperty):
        """
        Enforces the boundary condition of the specified type at location on the Grid

        Args:
            type (BCType): The type of the boundary condition
            location (BCLocation): Where to apply bc, if a list then the same bc is applied to all locations in the list
            val (float): value to set the gradient to
            fluid_property (FluidProperty): Which property to enforce the boundary condition on
        """
        match type:
            case BCType.DIRICHLET:
                if isinstance(location, list):
                    for loc in location:
                        self._bc_dirichlet(loc, val, fluid_property)        
                else:
                    self._bc_dirichlet(location, val, fluid_property)
                
            case BCType.NEUMANN:
                if isinstance(location, list):
                    for loc in location:
                        self._bc_neumann_0(loc, fluid_property)
                else:
                    self._bc_neumann_0(location, fluid_property)
    
    def _bc_dirichlet(self,
                      location: BCLocation, 
                      val: list[float,float] | float, 
                      fluid_property: FluidProperty):
        """
        Enforces the boundary condition on the grid

        Args:
            location (BCLocation): Where to apply the boundary condition
            val (list[float,float] | float): what to set the boundary to. expects a vector for velocity and a float for pressure
            fluid_property (FluidProperty): Which property to enforce the boundary condition on
        """
        if fluid_property == FluidProperty.VELOCITY:
            assert isinstance(val, list)
            self.ux[location.value] = val[0]
            self.uy[location.value] = val[1]

        elif fluid_property == FluidProperty.PRESSURE:
            assert isinstance(val, (float, int))
            self.p[location.value] = val

            
    def _bc_neumann_0(self,
                    location: BCLocation,
                    fluid_property: FluidProperty) -> None:
        """
        Enforces the boundary condition on the grid
        Neumann boundary condition  = 0
        Gradient at the boundary set to 0 
        -> value on the boundary same as value just inside the boundary
        Args:
            location (BCLocation): Where to apply bc
            fluid_property (FluidProperty): Which property to enforce the boundary condition on
        """
        if fluid_property == FluidProperty.VELOCITY:
            self.ux[location.value] = self.ux[location.inside().value]
            self.uy[location.value] = self.uy[location.inside().value]

        elif fluid_property == FluidProperty.PRESSURE:
            self.p[location.value] = self.p[location.inside().value]
            
            
    def plot_velocity(self) -> None:
        plt.quiver(self.x, self.y, self.ux, self.uy)
        plt.show()
        
    def plot_pressure(self) -> None:
        plt.imshow(self.p, cmap="hot",  interpolation="nearest")
        plt.show()
        
    def plot_contourf(self) -> None:
        raise NotImplementedError
        plt.contourf()

In [238]:
grid = Grid(GRID_SIZE, DOMAIN_SIZE)
grid.bc_enforce_pressure()
grid.bc_enforce_velocity()
grid_tmp = Grid(GRID_SIZE, DOMAIN_SIZE)

## 1. Impuls ohne Druckgradient

$$
\frac{\partial u}{\partial t} + (u * \nabla)u = \nu\nabla^2u
$$

1. for each point inside the domain (not on boundaries) calculate $\frac{\partial u}{\partial t}$ according to the equation above. That is u* not taking into account the pressure changes
2. Save the u* in a new grid 

### reminder for my stupid ass: np list slicing is [row_begin:row_end, column_begin:column_end]

In [239]:
def d_dx(grid: np.ndarray) -> np.ndarray:
    """
    central difference for all points inside the domain (not calculated on boundaries)
    val right - left

    Args:
        grid (np.ndarray): the grid with values for that the central difference in x direction should be performed

    Returns:
        np.ndarray: grid of same size as input, containing the central difference in x direction for each cell in the input
    """
    central_diff_x = np.zeros_like(grid)
    central_diff_x[:, 1:-1] = (grid[:, 2:] - grid[:, :-2]) / (2*h)
    return central_diff_x

def d_dy(grid: np.ndarray) -> np.ndarray:
    """
    central difference for all points inside the domain (not calculated on boundaries)
    val Bottom - Top

    Args:
        grid (np.ndarray): the grid with values for that the central difference in y direction should be performed

    Returns:
        np.ndarray: grid of same size as input, containing the central difference in y direction for each cell in the input
    """
    central_diff_y = np.zeros_like(grid)

    central_diff_y[1:-1, :] = (grid[2:,:] - grid[:-2,:]) / (2*h)
    return central_diff_y
    

def laplace(grid: np.ndarray) -> np.ndarray:
    """
    Laplace operator 
    applies the laplace operator to each cell inside the domain 
    (right + left + Bottom + Top - 4*center) / h^2

    Args:
        grid (np.ndarray): the grid where laplace should be applied

    Returns:
        np.ndarray: grid of same size as input, each cell containing the value obtained after applying the laplace operator to the input
    """
    lpc = np.zeros_like(grid)
    lpc[1:-1,1:-1] = (grid[1:-1, 2:] + grid[1:-1, :-2] + grid[2:,1:-1] + grid[:-2,1:-1] - 4*grid[1:-1,1:-1]) / (h**2)
    return lpc

In [240]:
def impulse_without_pressure_gradient(grid: Grid):
    dux_dt = nu*laplace(grid.ux)*grid.ux - (d_dx(grid.ux))*grid.ux
    duy_dt = nu*laplace(grid.uy)*grid.uy - (d_dy(grid.uy))*grid.uy
    return dux_dt, duy_dt

In [241]:
example = np.linspace(0,1,11)**2
example = [list(example*n) for n in range(11)]
example = np.array(example)
example[0,:] = 1
example[:,-1] = 0
example[:,-0] = 0
example[-1,:] = 0
assert round((0.04-0) / (h*2), 2) == round(d_dx(example)[1,1], 2)
assert round((0.02-1) / (h*2),2) == round(d_dy(example)[1,1], 2)
assert round((0.04 + 1.0 + 0.0 + 0.02 -4*0.01)/(h**2),2) == round(laplace(example)[1,1],2)

# print(example)
# print("\nAt 1,1")
# print(f"x diff: {(0.04-0) / (h*2)}")
# print(f"x diff func: {d_dx(example)[1,1]}")
# print(f"y diff: {(0.02-1) / (h*2)}")
# print(f"y diff func: {d_dy(example)[1,1]}")
# print(f"laplace: {(0.04 + 1.0 + 0.0 + 0.02 -4*0.01)/(h**2)}")
# print(f"laplace func: {laplace(example)[1,1]}")
# print(d_dx(example))
# print(d_dy(example))
# print(laplace(example))


In [242]:
dux_dt, duy_dt = impulse_without_pressure_gradient(grid)
grid_tmp.ux = dux_dt
grid_tmp.uy = duy_dt

### 1.1 Geschwindigkeit RB erzwingen

In [243]:
grid_tmp.bc_enforce_velocity()

### 2.1 RHSE der Druck Posson Gleichung

In [ ]:
(rho/delta_t) * (grid_tmp.ux + grid_tmp.uy)

array([[1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])